## Visualization of Schism results

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.tri as tri
from netCDF4 import Dataset
from scipy.io import loadmat
import pandas as pd
from scipy.interpolate import griddata


In [ ]:
from Poseidon.utils import *


In [ ]:
from matplotlib import animation,rc

rc('animation',html='html5')


In [ ]:
%matplotlib inline

In [ ]:
#g=loadmat('/home/critechproc/STORAGE/SCHISM/test1/nodes.mat')
#g['nodes'][:].shape

In [ ]:
#extract number of elements, number of nodes
ne,nn = pd.read_csv('/Users/brey/Downloads/windrot_geo2proj.gr3',header=0,skiprows=1,nrows=0,delim_whitespace=True)

In [ ]:
ne=int(ne)
nn=int(nn)

In [ ]:
#read lon,lat,depth for all nodes
q=pd.read_csv('/Users/brey/Downloads/windrot_geo2proj.gr3',skiprows=2,header=None,delim_whitespace=True,engine='python',nrows=nn,names=['id','x','y','z'])

In [ ]:
q=q.set_index(['id'])

In [ ]:
e = pd.read_csv('/Users/brey/Downloads/windrot_geo2proj.gr3',skiprows=nn+2,header=None,delim_whitespace=True,engine='python',nrows=ne,names=['id','nv','a','b','c'])

In [ ]:
e=e.set_index(['id'])

In [ ]:
r = Dataset('/Users/brey/Downloads/schism_history.nc')

In [ ]:
r.variables.keys()

In [ ]:
r.variables['WATLEV'].shape

## plot

In [ ]:
triangles = e.loc[:,['a','b','c']].values

In [ ]:
triangles

In [ ]:
triangles[:-2,:].max()

In [ ]:
x = q.loc[:,['x']].values.flatten()
y = q.loc[:,['y']].values.flatten()

In [ ]:
triang=tri.Triangulation(x,y)

In [ ]:
x.shape,y.shape

In [ ]:
z = q.loc[:,['z']].values.flatten()

In [ ]:
h = r.variables['WATLEV'][-1,:]

In [ ]:
minlon=-35. # lat/lon window
maxlon=43.
minlat=24.5
maxlat=76.5

In [ ]:
plt.figure(figsize=(12,10))
plt.gca().set_aspect('equal')
plt.tricontourf(triang, z, 50)
plt.colorbar()
plt.title('Contour plot')
plt.xlabel('Longitude (degrees)')
plt.ylabel('Latitude (degrees)')

In [ ]:
plt.figure(figsize=(12,10))
plt.gca().set_aspect('equal')
plt.tricontourf(x, y, triangles[:-2,:], h, 50, )
plt.colorbar()
plt.title('Contour plot')
plt.xlabel('Longitude (degrees)')
plt.ylabel('Latitude (degrees)')

## interpolating to Cartesian grid

In [ ]:
win={'lon0':x.min(), # lat/lon window of the figure
     'lon1':x.max(),
     'lat0':y.min(),
     'lat1':y.max()}

In [ ]:
xi = np.linspace(np.min(x), np.max(x), 100)
yi = np.linspace(np.min(y), np.max(y), 100)
zia = griddata((x,y),h,(xi[None,:],yi[:,None]),method='linear')

In [ ]:
lons,lats = np.meshgrid(xi,yi)

In [ ]:
map(lons,lats,zia,title='Storm Surge',label='m',ticks=False,**win)

## anim

In [ ]:
fig = plt.figure(figsize=(12,8))
ax=ax = plt.gca()
plt.gca().set_aspect('equal')

In [ ]:
ims = []
for i in range(len(h[1:,0])):
        im = ax.tricontourf(x, y, h[i,:], 50)
#        im = ax.contourf(x,y,z[i,:,:],v,vmin=v1,vmax=v2,latlon=True)
        add_arts = im.collections
        text = 'time={0!r}'.format(i)
        te = ax.text(90, 90, text)
        an = ax.annotate(text, xy=(-0.25, 0.95), xycoords='axes fraction')
        ims.append(add_arts + [te,an])
    
#cbar_ax = fig.add_axes([0.05, 0.05, 0.85, 0.05])    
#cbar = plt.colorbar(ticks=v,orientation='horizontal', extend='both')#,fraction=0.046, pad=0.04)
#cbar = plt.colorbar()

In [ ]:
anim = animation.ArtistAnimation(fig, ims, interval=200, blit=False,repeat=False)

In [ ]:
anim

## using mayavi

In [ ]:
from mayavi import mlab
#mlab.init_notebook('x3d')

In [ ]:
x.shape,triangles.shape

In [ ]:
mlab.triangular_mesh(x,y,h,triangles[:-2,:],scalars=h)

In [ ]:
mlab.show()